# Phase 0: PDF OCR with Position Tracking - Baseline

## Overview
This notebook implements the **baseline solution** provided by the Snowflake FCTO for extracting text from PDFs while capturing position information.

### What This Does:
- Extracts text from PDF documents stored in Snowflake stages
- Captures the **x,y coordinates** of each text box on the page
- Returns structured data: `{pos: (x,y), txt: text}`

### Customer Requirement This Addresses:
✅ **Document Intelligence - positioning capability** - knows where text appears on the page

### What's Missing (Future Phases):
- ❌ Page numbers
- ❌ Section detection
- ❌ Better chunking
- ❌ LLM integration
- ❌ Citation system

---


## Step 1: Environment Setup

Set up the Snowflake environment with appropriate roles and context.


In [ ]:
-- Use administrative role to grant permissions
USE ROLE accountadmin;


In [ ]:
-- Grant access to PyPI packages (needed for pdfminer library)
GRANT DATABASE ROLE SNOWFLAKE.PYPI_REPOSITORY_USER TO ROLE sysadmin;


In [ ]:
-- Switch to sysadmin role for creating objects
USE ROLE sysadmin;


## Step 2: Database and Schema Setup

Create or use an existing database and schema for this project.


In [ ]:
-- Create database if it doesn't exist
CREATE DATABASE IF NOT EXISTS pdf_ocr_demo;
USE DATABASE pdf_ocr_demo;


In [ ]:
-- Create schema if it doesn't exist
CREATE SCHEMA IF NOT EXISTS public;
USE SCHEMA public;


## Step 3: Create Stage for PDF Storage

Stages in Snowflake are locations where data files are stored. We'll create an internal stage to hold our PDF documents.


In [ ]:
-- Create internal stage for PDF files
CREATE STAGE IF NOT EXISTS pdf
COMMENT = 'Stage for storing clinical protocol PDFs and other documents';


In [ ]:
-- Verify stage was created
SHOW STAGES LIKE 'pdf';


## Step 4: Create PDF Text Mapper UDF

This User-Defined Function (UDF) is the core of our solution. Let's break down what it does:

### Technology Stack:
- **Language:** Python 3.12
- **Library:** `pdfminer` - A robust PDF parsing library
- **Snowflake Integration:** Uses `SnowflakeFile` to read directly from stages

### How It Works:
1. Opens the PDF file from the Snowflake stage
2. Iterates through each page
3. Extracts text boxes (`LTTextBox` objects) from the page layout
4. Captures the **bounding box coordinates** (bbox) - specifically:
   - `bbox[0]` = x-coordinate (left)
   - `bbox[3]` = y-coordinate (top)
5. Returns an array of objects: `{pos: (x,y), txt: text}`

### Input:
- `scoped_file_url`: A Snowflake-generated URL pointing to a file in a stage

### Output:
- VARCHAR (JSON string) containing array of text boxes with positions


In [ ]:
CREATE OR REPLACE FUNCTION pdf_txt_mapper(scoped_file_url string)
RETURNS VARCHAR
LANGUAGE PYTHON
RUNTIME_VERSION = '3.12'
ARTIFACT_REPOSITORY = snowflake.snowpark.pypi_shared_repository
PACKAGES = ('snowflake-snowpark-python', 'pdfminer')
HANDLER = 'main'
AS
$$
from snowflake.snowpark.files import SnowflakeFile
from pdfminer.layout import LAParams, LTTextBox
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator

def main(scoped_file_url):
    finding = []
    with SnowflakeFile.open(scoped_file_url, 'rb') as f:
        # Initialize PDF processing components
        rsrcmgr = PDFResourceManager()
        laparams = LAParams()  # Layout analysis parameters
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        pages = PDFPage.get_pages(f)
        
        # Process each page
        for page in pages:
            interpreter.process_page(page)
            layout = device.get_result()
            
            # Extract text boxes from the page
            for lobj in layout:
                if isinstance(lobj, LTTextBox):
                    # bbox = (x0, y0, x1, y1) where (x0,y0) is bottom-left, (x1,y1) is top-right
                    x, y, text = lobj.bbox[0], lobj.bbox[3], lobj.get_text()
                    finding += [{'pos': (x, y), 'txt': text}]
    
    return str(finding)
$$;


In [ ]:
-- Verify function was created
SHOW FUNCTIONS LIKE 'pdf_txt_mapper';


## Step 5: Upload PDF to Stage

### Instructions:

**Option 1: Using SnowSQL CLI**
```bash
snowsql -a <account> -u <username>
PUT file:///Users/akelkar/src/Cursor/pdf-ocr-with-position/Prot_000.pdf @pdf_ocr_demo.public.pdf AUTO_COMPRESS=FALSE;
```

**Option 2: Using Snowflake Web UI**
1. Navigate to Data → Databases → PDF_OCR_DEMO → PUBLIC → Stages
2. Click on the `PDF` stage
3. Click "+ Files" button in the top right
4. Upload `Prot_000.pdf`

**Option 3: Using Python Snowpark**
```python
session.file.put("Prot_000.pdf", "@pdf", auto_compress=False)
```

Let's verify the file after upload:


In [ ]:
-- List files in the PDF stage
LIST @pdf;


## Step 6: Test the PDF Text Mapper

Now let's test our function with the uploaded PDF.

### What to Expect:
- The function will return a VARCHAR (string representation of a Python list)
- Each element will be: `{'pos': (x, y), 'txt': 'extracted text'}`
- The output will be **very long** for multi-page documents

### Note on `build_scoped_file_url()`:
This Snowflake function generates a temporary, scoped URL that allows the UDF to securely access the staged file.


In [ ]:
-- Test with the clinical protocol PDF
-- This will return the full extracted text with positions
SELECT pdf_txt_mapper(build_scoped_file_url(@pdf, 'Prot_000.pdf')) AS extracted_data;


## Step 7: Analyze the Output

Let's get some basic statistics about what was extracted.


In [ ]:
-- Get the length of the output
SELECT 
    LENGTH(pdf_txt_mapper(build_scoped_file_url(@pdf, 'Prot_000.pdf'))) AS output_length_chars,
    LENGTH(pdf_txt_mapper(build_scoped_file_url(@pdf, 'Prot_000.pdf'))) / 1024 AS output_length_kb;


## Phase 0 Summary

### ✅ What We've Accomplished:
1. Set up Snowflake environment with proper roles and permissions
2. Created a stage for storing PDF documents
3. Deployed the FCTO's baseline PDF text mapper UDF
4. Extracted text from a clinical protocol PDF with position information

### 📊 Current Output Format:
```python
[{'pos': (54.0, 720.3), 'txt': 'CLINICAL PROTOCOL\n'}, 
 {'pos': (72.0, 680.1), 'txt': 'Study Title: ...\n'},
 ...]
```

### 🎯 What This Gives Us:
- ✅ Text extraction from PDFs
- ✅ X,Y coordinates for each text box
- ✅ Snowflake-native processing (no external services)

### ⚠️ Current Limitations:
- ❌ No page number information
- ❌ No section/hierarchy detection
- ❌ Text boxes may be too granular or broken
- ❌ Output is a string, not structured data we can query
- ❌ No way to answer "Where did this info come from?"

---

## Next Steps: Phase 1
In the next phase, we'll enhance this solution to:
1. **Add page numbers** to each text box
2. Store results in a **queryable table** instead of a string
3. Add a **unique chunk ID** for each text box

This will enable queries like:
```sql
SELECT * FROM document_chunks 
WHERE page = 5 
AND txt ILIKE '%medication%';
```


## Troubleshooting

### Common Issues:

**1. Permission Error on PyPI:**
```
Error: Access denied for database role SNOWFLAKE.PYPI_REPOSITORY_USER
```
**Solution:** Make sure you ran the GRANT command as ACCOUNTADMIN

**2. File Not Found:**
```
Error: File 'Prot_000.pdf' does not exist
```
**Solution:** Verify the file was uploaded with `LIST @pdf;`

**3. Function Takes Too Long:**
- Large PDFs (100+ pages) can take 30-60 seconds
- This is normal for the initial processing
- Consider processing in batches for very large documents

**4. Memory Issues:**
- For very large PDFs (500+ pages), you may need to increase warehouse size
- Or split the PDF into smaller chunks before processing
